-Getting player data since 2000 from Season Stats CSV
-Getting team roster 

In [94]:
#Libraries/Imports

import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [207]:
allLinks = []

for year in range(2000,2022):
    url = "https://www.basketball-reference.com/leagues/NBA_{}.html".format(year)

    response = requests.request("GET", url).text
    soupStruct = bs(response)

    allTables = soupStruct.findAll('table')[:2]
    
    for table in allTables:
        for row in table.find_all('tr'):
            for th in row.find_all('th'):
                aTag = th.a
                if aTag != None:
                    allLinks.append(aTag['href'])
print(allLinks)

['/teams/MIA/2000.html', '/teams/NYK/2000.html', '/teams/PHI/2000.html', '/teams/ORL/2000.html', '/teams/BOS/2000.html', '/teams/NJN/2000.html', '/teams/WAS/2000.html', '/teams/IND/2000.html', '/teams/CHH/2000.html', '/teams/TOR/2000.html', '/teams/DET/2000.html', '/teams/MIL/2000.html', '/teams/CLE/2000.html', '/teams/ATL/2000.html', '/teams/CHI/2000.html', '/teams/UTA/2000.html', '/teams/SAS/2000.html', '/teams/MIN/2000.html', '/teams/DAL/2000.html', '/teams/DEN/2000.html', '/teams/HOU/2000.html', '/teams/VAN/2000.html', '/teams/LAL/2000.html', '/teams/POR/2000.html', '/teams/PHO/2000.html', '/teams/SEA/2000.html', '/teams/SAC/2000.html', '/teams/GSW/2000.html', '/teams/LAC/2000.html', '/teams/PHI/2001.html', '/teams/MIA/2001.html', '/teams/NYK/2001.html', '/teams/ORL/2001.html', '/teams/BOS/2001.html', '/teams/NJN/2001.html', '/teams/WAS/2001.html', '/teams/MIL/2001.html', '/teams/TOR/2001.html', '/teams/CHH/2001.html', '/teams/IND/2001.html', '/teams/DET/2001.html', '/teams/CLE/200

In [211]:
rosterData = pd.DataFrame(data = None, index = None, columns = None, dtype = None, copy = None)

for link in allLinks:
    #creating the link with array with combo of links and years
    url = "https://www.basketball-reference.com" + link
    
    #sending request
    response = requests.request("GET", url).text
    soup = bs(response)
    
    #finding the record by looking for the word "Record:"
    record = soup.find(lambda tag: tag.name == "strong" and "Record:" in tag.text).next_sibling 
    winsLoss = record.split(',')[0].strip().split('-') #looking for wins
    wins = int(winsLoss[0])
    losses = int(winsLoss[1])
    
    #finding the correct table from the team website by looking for keywords "Totals"
    rosterTable = soup.find(lambda tag: tag.name == "h2" and "Advanced" in tag.text).find_next("table")
    
    #converting from html to dataframe
    teamRosterDF = pd.read_html(str(rosterTable))[0]
    
    #adding the year,team, wins, and losses to the dataframe
    teamRosterDF["Year"] = url[-9:-5]
    teamRosterDF["Team"] = url[-13:-10]
    teamRosterDF["Wins"] = wins
    teamRosterDF["Losses"] = losses
    teamRosterDF.rename(columns = {'Unnamed: 1':'Name'}, inplace = True) #renaming the name column
    rosterData = rosterData.append(teamRosterDF) #adding to the complete roster data
rosterData = rosterData.dropna(subset=['Name']) 
rosterData

,Rk,Name,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,...,WS/48,Unnamed: 22,OBPM,DBPM,BPM,VORP,Year,Team,Wins,Losses
0,1,Jamal Mashburn,27,76,2828,15.4,0.526,0.240,0.206,2.8,...,0.109,NaN,0.9,-0.4,0.5,1.8,2000,MIA,52,30
1,2,Alonzo Mourning,29,79,2748,25.8,0.596,0.003,0.492,9.5,...,0.226,NaN,3.1,2.2,5.3,5.0,2000,MIA,52,30
2,3,Dan Majerle,34,69,2308,11.4,0.559,0.720,0.164,1.4,...,0.126,NaN,-0.2,2.1,1.9,2.3,2000,MIA,52,30
3,4,P.J. Brown,30,80,2302,15.3,0.516,0.001,0.237,11.4,...,0.143,NaN,-0.3,0.6,0.3,1.4,2000,MIA,52,30
4,5,Anthony Carter,24,79,1859,11.0,0.442,0.045,0.244,3.1,...,0.036,NaN,-3.0,0.2,-2.7,-0.4,2000,MIA,52,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26,Rodions Kurucs,22,11,75,0.8,0.297,0.714,0.095,2.7,...,-0.176,NaN,-10.1,1.5,-8.6,-0.1,2021,HOU,17,55
26,27,Bruno Caboclo,25,6,36,9.0,0.475,0.294,0.118,8.5,...,-0.088,NaN,-8.5,-1.7,-10.2,-0.1,2021,HOU,17,55
27,28,Cameron Reynolds,25,2,34,6.5,0.406,0.750,0.000,6.0,...,-0.043,NaN,-3.4,-4.8,-8.2,-0.1,2021,HOU,17,55
28,29,Brodric Thomas,24,4,24,7.2,0.496,0.857,1.000,4.3,...,-0.063,NaN,-7.0,0.5,-6.5,0.0,2021,HOU,17,55


In [214]:
csv = rosterData.to_csv("all_player_data.csv")